In [1]:
import os
import sys
sys.path.append(f"{os.getcwd()}/../src")

## Setup Params

In [3]:
from data_collection.random_cpc import random_code_trials, RandomCPCTrialParams

trials = []
p_error_rates = [[0.005, 0, 0.005], [0.01, 0, 0.01],
                 [0.015, 0, 0.015], [0.02, 0, 0.02]]
ns = [20, 40, 80, 120]
mxs = mzs = [15, 30, 60, 90]
dvx = dvz = 3
n_cross_checks = [30, 90, 150, 210]  # TODO: this is totally random!!

for p in p_error_rates:
    for i in range(len(ns)):
        trials.append(RandomCPCTrialParams(
            ns[i], mxs[i], dvx, mzs[i], dvz, n_cross_checks[i], p, seeds=[0, 1]))

random_code_trials(trials, "../saved_cpc_codes", "../eval/random_res.json")


Simplifying the input code
Simplifying the input code
Simplifying the input code


KeyboardInterrupt: 